# MapReduce with mrjob

In [1]:
import json

In [2]:
data =[]
with open('yelp_review_subset.json') as f:
    for line in f:
        data.append(json.loads(line))

In [6]:
data[0]["text"]

'Mr Hoagie is an institution. Walking in, it does seem like a throwback to 30 years ago, old fashioned menu board, booths out of the 70s, and a large selection of food. Their speciality is the Italian Hoagie, and it is voted the best in the area year after year. I usually order the burger, while the patties are obviously cooked from frozen, all of the other ingredients are very fresh. Overall, its a good alternative to Subway, which is down the road.'

In [35]:
textfalse = ''
for x in data:
    if x['review_id'] == "xq1Y9AFMEKFEmqhAJDS_Gw":
        textfalse = ((x['text']))

In [25]:

textright = ''
for x in data:
    if x['review_id'] == "6O1CBD0c9Zm7siaK22vibA":
        textright = ((x['text']))

In [27]:
import string
exclude = set(string.punctuation)

In [36]:
setfalse = {}
textfalse = ''.join(textfalse.split('\n'))

textfalse = ''.join(ch for ch in textfalse if ch not in exclude)

textfalse = textfalse.split(' ')

for t in textfalse:
    term = t.lower()
    if term not in setfalse:
        setfalse[term] = 1
    else:
        setfalse[term] += 1




In [40]:
set2 = []
for k,v in setfalse.items():
    if v==1:
        set2.append(k)

In [41]:
len(set2)

220

In [42]:
setright = {}
textright = ''.join(textright.split('\n'))

textright = ''.join(ch for ch in textright if ch not in exclude)

textright = textright.split(' ')

for t in textright:
    term = t.lower()
    if term not in setright:
        setright[term] = 1
    else:
        setright[term] += 1




In [43]:
set1 = []
for k,v in setright.items():
    if v==1:
        set1.append(k)

In [44]:
len(set1)

390

In [45]:
set1

['played',
 'kitchen',
 'up',
 'rebranded',
 'courses',
 'allowed',
 'highlighted',
 'interesting',
 'personally',
 'olive',
 'risotto',
 'recognition',
 'defining',
 'satisfaction',
 'you',
 'indulge',
 'also',
 'beneath',
 'choose',
 'single',
 'rather',
 'plating',
 'gem',
 'entrees',
 'an',
 'stripside',
 'thigh',
 'cream',
 'group',
 'bright',
 'arriving',
 'undersaucing',
 'order',
 'tasting',
 'known',
 'around',
 'lightest',
 'round',
 'indicated',
 'family',
 'marketing',
 'lustinducing',
 'types',
 'upper',
 'familystyle',
 'playing',
 'show',
 'friendly',
 'coated',
 'punctuated',
 'fonduta',
 'reportedly',
 'seemingly',
 'agnolotti',
 'past',
 'cursive',
 'music',
 'seared',
 'steak',
 'quintet',
 'yearnot',
 'flavors',
 'any',
 'mozzarella',
 'lightly',
 'its',
 'brand',
 'focaccia',
 'spicy',
 'pervasive',
 'gnocchi',
 'putting',
 'miles',
 'hash',
 'know',
 'progressing',
 'garlic',
 'undoubtedly',
 'strong',
 'days',
 'al',
 'bone',
 'out',
 'fettuccini',
 'tucked',
 'w

In this assignment, we will write multi-step MapReduce jobs Python using [mrjob](https://pythonhosted.org/mrjob/). First, we will test it on our local machine and then run it on Amazon AWS.

We will use the [Yelp reviews dataset](https://drive.google.com/a/bu.edu/file/d/0B4bdB5WPHGsqcmJMNmxzQmQ3aTQ/view?usp=sharing) from Assignment 2.

This assignment should be completed **individually**.

----------------------------------------

## Apply for AWS Educate Account (do this now!)

It is extremely important that you apply for an "AWS Educate" account right away to get $100 free credit, and verify that the credit has been properly applied to your account.

* Go to https://www.awseducate.com/Application
* Choose **Student** and click **Next**
* Fill out the application

We recommend selecting an AWS Educate Starter Account. If, instead, you want to use an AWS Account ID, it is important that you set up a billing alarm on AWS to notify you when your credits are running low.

Remember to shut down **everything** when you're done with the instances, or you may get surprising bills.

----------------------------------------

## Task 1: Word Count

Write a mapper and reducer that yield the total uses of a word across all reviews. (**5pts**)

In the map step, we want to extract words from the text of reviews. Then, count the words in each review. In the reduce step, summarize all the counts by taking the sum.

A recap of how to use mrjob: https://pythonhosted.org/mrjob/guides/quickstart.html

Note, when you are testing it on a local machine, you may only want to run a subset of the dataset.

In [ ]:
from mrjob.job import MRJob
from mrjob.step import MRStep
from mrjob.protocol import JSONValueProtocol
import re, string

WORD_RE = re.compile(r"[\w']+")

class MRCount(MRJob):

    #INPUT_PROTOCOL = JSONProtocol
    INPUT_PROTOCOL = JSONValueProtocol
    OUTPUT_PROTOCOL = JSONValueProtocol

    def mapper(self, key, review):

        # punctuation set
        exclude = set(string.punctuation)

        # remove the new line character from the string
        text = ''.join(review['text'].split('\n'))

        # remove all puncuation from the string 
        text = ''.join(ch for ch in text if ch not in exclude)

        # split the text
        text = text.split(' ')

        for term in text: 
            yield term.lower(), 1 

        

    def reducer(self, term, counts):
        yield None, {'term': term, 'count': sum(counts)}







if __name__ == '__main__':
    MRMostUsedWord.run()



## Task 2: Unique Review

Write a multi-step map reduce that yield the review_id with text containing the maximum number of unique words across all reviews. (**15pts**)

In [ ]:
from mrjob.job import MRJob 
from mrjob.step import MRStep
from mrjob.protocol import JSONValueProtocol
import re, string

# Write a multi-step map reduce that yield the review_id with text 
# containing the maximum number of unique words across all reviews.


class MRMostUsedWord(MRJob):
    INPUT_PROTOCOL = JSONValueProtocol
    OUTPUT_PROTOCOL = JSONValueProtocol
    def steps(self):
        return [
            MRStep(mapper=self.mapper_id_words),
                   #combiner=self.combiner_count_words,
                   #reducer=self.reducer_count_words),
            MRStep(reducer=self.reducer_id_words),
            MRStep(reducer=self.reducer_unique_word_list),
            MRStep(reducer=self.reducer_find_max_unique_review)
        ]

    def mapper_id_words(self, key, review):
    	exclude = set(string.punctuation)

    	review_id = review['review_id']

    	text = ''.join(review['text'].split('\n'))

    	text = ''.join(ch for ch in text if ch not in exclude)

    	text = text.split(' ')

    	for term in text:
    		# yield (id, word), 1
            
            if(term!=""):
                #print(review_id, term)
                yield (review_id, term.lower()),1

    def reducer_id_words(self, key, counts):
        c = sum(counts)
        if c == 1:
            yield key[0], c

    def reducer_unique_word_list(self, key, counts):
        c = sum(counts)
        #print(key,c)
        yield None, (c,key)

    def reducer_find_max_unique_review(self, _, counts):
        yield max(counts)




if __name__ == '__main__':
    MRMostUsedWord.run()




## Task 3: User Similarity

Write a map reduce that measures the Jaccard similarity of users based on the businesses reviewed, and find users with >= 0.5 similarity. (**15pts**)

In [ ]:
from mrjob.job import MRJob 
from mrjob.step import MRStep
from mrjob.protocol import JSONValueProtocol
import re, string

# Write a map reduce that measures the Jaccard similarity of users based on 
# the businesses reviewed, and find users with >= 0.5 similarity. 


class MRMostUsedWord(MRJob):

    INPUT_PROTOCOL = JSONValueProtocol
    OUTPUT_PROTOCOL = JSONValueProtocol
    
    def steps(self):
        return [
            MRStep(mapper=self.mapper_uid_bid),
                   #combiner=self.combiner_count_words,
                   #reducer=self.reducer_uid_list),
            MRStep(reducer=self.reducer_uid_bs_list),
            MRStep(reducer=self.reducer_bid_to_uid),
            MRStep(reducer=self.reducer_pair),
            MRStep(reducer=self.reducer_remove_duplicate)
        ]

    def mapper_uid_bid(self, key, review):
        user_b_list = {}
        user_id = review['user_id']
        business_id = review['business_id']

        # update the dictionary
        if user_id in user_b_list:
            user_b_list[user_id].append(business_id)
        else:
            l = []
            l.append(business_id)
            user_b_list[user_id] = l
        
        yield user_id, business_id


    def reducer_uid_bs_list(self, user_id, business_id_list):
        bid_list = []
        for bid in business_id_list:
            bid_list.append(bid)

        yield user_id, bid_list


    def reducer_bid_to_uid(self, user_id, business_id_list):
        bid_list = []
        for item in business_id_list:
            for bid in item:
                bid_list.append(bid)

        for bid in bid_list:

            yield bid, (user_id, bid_list)




    def reducer_pair(self, bid, li):
        business_list = []
        for x in li:
            business_list.append(x)

        length = len(business_list)

        # sorte the business_list, prevent when do the double
        business_list = sorted(business_list, key=lambda x: x[0])

        i = 0
        for i in range(0,length):
            j = i+1
            if j<length:
                for k in range(j,length):
                    #print(business_list[i])
                    #print(business_list[i])
                    s1 = set(business_list[i][1])
                    s2 = set(business_list[k][1])

                    score = len(s1.intersection(s2))/len(s1.union(s2))
                    pair1 = business_list[i][0]
                    pair2 = business_list[k][0]

                    #yield set((pair1,pair2)), {'pair':pair1+' '+pair2, 'score':score}
                    if score>0.5:
                        yield (pair1,pair2), score

    def reducer_remove_duplicate(self, pair, score):
        l = []
        for x in score:
            l.append(x)
        yield None, {'pair':pair, 'score':l[0]}


if __name__ == '__main__':
    MRMostUsedWord.run()

----------------------------------------

## Task 4: Run on Amazon Elastic MapReduce

We will be using the Amazon Simple Storage Service (S3). To create an S3 bucket, sign into the AWS Management Console and open the Amazon S3 console at https://console.aws.amazon.com/s3. Click **Create Bucket** on the top left. After you have created your bucket, click **Upload** and upload yelp_academic_dataset_review.json in your bucket.

To allow mrjob to run your code on Amazon EMR, first you need to configure your AWS and SSH credentials: http://pythonhosted.org/mrjob/guides/emr-quickstart.html

The following command will read in yelp_academic_dataset_review.json from your S3 bucket and store the output in a directory called "output" in your S3 bucket (you can view your S3 bucket at https://console.aws.amazon.com/s3). The output will also be streamed back to your local machine.

`python unique_review.py -r emr s3://name-of-your-bucket/yelp_academic_dataset_review.json --output-dir=s3://name-of-your-bucket/output/`

By default, mrjob runs a single m1.medium, which is a cheap but not very powerful instance type. To get more performance out of your job, you can eiher add more instances, use more powerful instances, or both. Amazon bills you for the full hour even if your cluster only lasts for a few minutes.

The basic way to control type and number of instances is with `instance_type` and `num_core_instances` options on the command line like this:

`--instance_type m1.medium --num-core-instances 4`

or adding to mrjob.conf that you created earlier:

`runners:
  emr:
    instance_type: m1.medium
    num_core_instances: 4
`

You can see the jobs you run here: http://console.aws.amazon.com/elasticmapreduce/

Here are a few more links that you may find useful:
* https://aws.amazon.com/emr/pricing/
* http://docs.aws.amazon.com/AWSEC2/latest/UserGuide/using-vpc.html#vpc-only-instance-types
* http://docs.aws.amazon.com/ElasticMapReduce/latest/DeveloperGuide/emr-troubleshoot-failed.html
* http://docs.aws.amazon.com/ElasticMapReduce/latest/DeveloperGuide/emr-troubleshoot-slow.html

Submit the output of your 3 tasks running on Amazon EMR. (**15pts**)

You can redirect the output to a file with the following:

`python unique_review.py -r emr s3://name-of-your-bucket/yelp_academic_dataset_review.json --output-dir=s3://name-of-your-bucket/output/ > unique_review.out`

----------------------------------------